In [ ]:
import torch
import import_ipynb
from main import NewsTechLSTM

model = NewsTechLSTM()
model.load_state_dict(torch.load("best_news_tech_model.pt"))
model.eval()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


            Date                                    sentence_vector
0     2008-01-02  [-0.7761599562784074, 0.47997251415681624, -0....
1     2008-01-03  [-0.6347472432343836, 0.44354152362158195, -0....
2     2008-01-04  [-0.7335159517129167, 0.4504950059141669, -0.1...
3     2008-01-07  [-0.3853699918656138, 0.4493455685154058, -0.0...
4     2008-01-08  [-0.4384423515390317, 0.4678929434299644, -0.0...
...          ...                                                ...
1505  2013-12-23  [-0.18712536263399046, 0.5489453625566331, 0.0...
1506  2013-12-24  [-0.07440841145496566, 0.4141136289749069, 0.2...
1507  2013-12-26  [-0.014801803165466415, 0.3908982870325198, 0....
1508  2013-12-27  [-0.3035191169999097, 0.4641025254924536, 0.10...
1509  2013-12-30  [-0.15629229510883635, 0.5007521043413246, 0.1...

[1510 rows x 2 columns]
0       2008-01-02
1       2008-01-03
2       2008-01-04
3       2008-01-07
4       2008-01-08
           ...    
1505    2013-12-23
1506    2013-12-24
1507   

<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
<string>:15: FutureWarning: Calling float on a single element Se

News shape: (1489, 5, 100)
Tech shape: (1489, 5, 7)
Labels shape: (1489,)

--- Fold 1 ---
